In [1]:
import warnings
warnings.filterwarnings('ignore')

# CS985: Assignment 1

## Problem Statement

I wished to explore the problem of using data about a listing to predict its price. Whilst not the most imaginative problem (compared to something like predicting the number of bathrooms a listing has using its postcode), it is realistic. For example, a price predictor could be used by a host to figure out what a fair price would be or by Airbnb to let users sort listings by value (calculated by the difference between the predicted and actual prices). My plan is to use as many different models as possible and compare the error in their predictions.

## Summarisation

I imported pandas and read the data set into a DataFrame, a 2-dimensonal labeled data structure.

In [ ]:
import pandas as pd
df = pd.read_csv("BristolAirbnbListings.csv")

The best thing to do when faced with any data analysis problem is to look at the big picture. pandas has a few helpful functions like info(), head() and describe() that can be used to gain insights.

shape yields the height and width.

In [ ]:
df.shape

info() prints the number of rows and columns, the name and type of each column and the number of non-null values in each column.

In [ ]:
df.info()

My first impression was that there was a lot of data about each listing, only some of which would be helpful. Most of the column names were self-explanatory.

describe() prints the mean, standard deviation, minimum, 25th, 50th and 75th percentiles and maximum of each numerical column.

In [ ]:
df.describe()

I made lots of observations off the back of this. For example, half of the listings were priced between £35 and £85 and the majority could be booked for as few as one or two nights.

(I changed the value of display.max_columns to None. This option decides how many columns to display. If Python is running in a terminal, the default value is 0, which means "display as many columns as possible with respect to the width of the terminal". In other contexs, the default value is 20. None means "print all the columns".)

In [ ]:
pd.set_option('display.max_columns', None)

head() prints the first 5 rows.

In [ ]:
df.head()

This showed me some example listings. Something I didn't understand was why some columns were floats (like latitude) or ints (like price) but other columns that ought to be numbers (like accommodates) were "objects".

I used value_counts() to explore this. value_counts() returns counts of unique values. In other words, it returns each different value in a column and the number of times it occurs.

In [ ]:
df["accommodates"].value_counts()

The accommodates column contains ints like 2 and 4 but there is some junk like Private room and Entire home/apt stopping it from being a column of ints. The solution to this is cleaning. Cleaning means detecting and correcting (or removing) corrupt or inaccurate records.

## Visualisation

I used matplotlib to plot some of the data. Visualisation is a good way to understand what the data "looks like". [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet) is an example of why visualisation is important.

In [ ]:
import matplotlib.pyplot as plt

First, I plotted a "map" of dots using latitude and longitude.

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude")
plt.show()

Second, I made each dot translucent to discover denser and sparser neighbourhoods.

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.2)
plt.show()

Third, I used the price of each listing to control its dot's size and colour. I made a copy of the DataFrame excluding any listings with price > 500 (the listing with price 5000 was causing almost every other listing to be basically the same shade of blue).

In [ ]:
df2 = df.copy()
for x in range(df2.shape[0]):
    if df2["price"][x] > 500:
        df2 = df2.drop(x)       
df2 = df2.reset_index(drop=True)

df2.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=df2["price"]/10, label="price", figsize=(10,7),
    c="price", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()
plt.show()

Some observations I made were

* There are more listings closer to the centre
* Listings closer to the centre are a bit more expensive
* The most expensive listings (yellow, orange and red) can be found both in the centre and on the outskirts
* The vast majority of listings cost £100 or less

## Preparation

Before I went any further, I dropped some columns.

* id, name, host_id and host_name because they don't matter
* neighbourhood and postcode because latitude and longitude are enough
* minimum_nights because almost every listing has 1 <= minimum_nights <= 2
* last_review because it doesn't matter
* review_scores_* because almost every listing has 9 <= review_scores_* <= 10
* calculated_host_listings_count because it's about the host, not the listing

In [ ]:
df = df.drop(["id", "name", "host_id", "host_name", "neighbourhood", "postcode", "minimum_nights", "last_review", "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value", "calculated_host_listings_count"], axis=1)

I checked which columns had any null values.

In [ ]:
df.isnull().any()

I iterated over each value in the accommodates column and weeded out any non-ints.

In [ ]:
for x in range(df.shape[0]):
    if not df["accommodates"][x].isdigit():
        df = df.drop(x)

There are two side effects. The first is that even though there are no non-int values in the accommodates column, it's still a column of objects, not ints. I changed the type of the accommodates column to int.

In [ ]:
df["accommodates"] = df["accommodates"].astype(int)

The second is that indices don't automatically change when rows are dropped. What this means is that if the xth row was dropped, the index x wouldn't point to anything. Another way to think about this is that the for loop used to iterate over each row (for x in range(df.shape[0])) wouldn't work because there would be some rows with indices greater than the number of rows. I solved this problem with reset_index(), making sure to drop the old indices (which would otherwise be preserved in new column).

In [ ]:
df = df.reset_index(drop=True)

I prepared bathrooms, bedrooms and beds in the same way but first I addressed the null values in those columns by dropping the rows.

In [ ]:
df = df.dropna(subset=["bathrooms", "bedrooms", "beds"])
df = df.reset_index(drop=True)

I made sure I didn't drop too many rows.

In [ ]:
df.shape[0]

In [ ]:
df["bathrooms"].value_counts()

In [ ]:
df["bathrooms"] = df["bathrooms"].astype(float)
df["bedrooms"].value_counts()

In [ ]:
df["bedrooms"] = df["bedrooms"].astype(int)
df["beds"].value_counts()

In [ ]:
df["beds"] = df["beds"].astype(int)
df.info()

Happily, accommodates, bathrooms, bedrooms and beds were all ints/floats. The significance of this was that they could now be used to predict price. I scanned the rest of the columns and found property_type and room_type. It was intuitive that these would correlate with price but there was a problem.

In [ ]:
df["property_type"].value_counts()

In [ ]:
df["room_type"].value_counts()

Text attributes like these can't be easily handled by machine learning algorithms. The most basic option is to convert this from text to numerical data. For example, Apartment = 1, House = 2, etc. This may suggest relationships between values that don't exist. For example, a Bungalow is not "nine times" an Apartment. One-hot encoding comes to the rescue. The column to be one-hot encoded is expanded to an array with one binary attribute per category.

I didn't want to introduce new columns for things like Boutique hotel and Tent so I shrunk property_type to three values.

* anything with "house" in it -> "House"
* anything with "apartment" in it -> "Apartment"
* everything else -> "Random"

In [ ]:
for x in range(df.shape[0]):
    if df["property_type"][x] == "Townhouse" or df["property_type"][x] == "Guesthouse" or df["property_type"][x] == "Tiny house":
        df.at[x, "property_type"] = "House"
    elif df["property_type"][x] == "Serviced apartment":
        df.at[x, "property_type"] = "Apartment"
    if df["property_type"][x] != "House" and df["property_type"][x] != "Apartment":
        df.at[x, "property_type"] = "Other"
df["property_type"].value_counts()

## Correlation

The correlation between two variables is the strength of the linear relationship between them. For example, if x = y, the correlation between x and y is 1 (perfect correlation). I explored the correlation between price and some other variables I expected to correlate with it and plotted this as a scatter matrix.

In [ ]:
corr_matrix = df.corr()
corr_matrix
corr_matrix["price"].sort_values(ascending=False)

from pandas.plotting import scatter_matrix
attributes = ["price", "accommodates", "bathrooms",
              "bedrooms", "beds"]
scatter_matrix(df[attributes], figsize=(12,7))
plt.show()

I also plotted price's correlation with each of accommodates, bathrooms, bedrooms and beds and adjusted the scales to zoom in on most of the listings.

In [ ]:
df.plot(kind="scatter", x="accommodates", y="price", alpha=0.1)
plt.axis([0, 10, 0, 400])
plt.show()

In [ ]:
df.plot(kind="scatter", x="bathrooms", y="price", alpha=0.1)
plt.axis([0, 4, 0, 400])
plt.show()

In [ ]:
df.plot(kind="scatter", x="bedrooms", y="price", alpha=0.1)
plt.axis([0, 6, 0, 400])
plt.show()

In [ ]:
df.plot(kind="scatter", x="beds", y="price", alpha=0.1)
plt.axis([0, 6, 0, 400])
plt.show()

## One-hot Encoding and Scaling

I one-hot encoded the property_type and room_type columns. At the same time, I scaled the rest of the data. Scaling is important because machine learning algorithms often don't perform well when the numerical attributes have very different scales. I used StandardScaler() twhich scales by standardisation (as opposed to min-max scaling). Finally, I combined the categorical and numerical data.

In [ ]:
property_cat = df["property_type"]
room_cat = df["room_type"]

property_cat_encoded, property_categories = property_cat.factorize()
room_cat_encoded, room_categories = room_cat.factorize()

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
property_cat_1hot = encoder.fit_transform(property_cat_encoded.reshape(-1,1))
room_cat_1hot = encoder.fit_transform(room_cat_encoded.reshape(-1,1))

pcea = property_cat_1hot.toarray()
rcea = room_cat_1hot.toarray()

df_num = df.drop(["property_type", "room_type"], axis=1)

cat_attribs = ["property_type", "room_type"]

num_attribs = list(df_num)

from sklearn import preprocessing
std_scaler = preprocessing.StandardScaler()
df_num[num_attribs] = std_scaler.fit_transform(df_num[num_attribs])

property_enc_data = pd.DataFrame(property_cat_1hot.toarray())
property_enc_data.columns = property_categories
property_enc_data.index = df.index

room_enc_data = pd.DataFrame(room_cat_1hot.toarray())
room_enc_data.columns = room_categories
room_enc_data.index = df.index

df_prepared = df_num.join([property_enc_data, room_enc_data])

(df is easier to type than df_prepared.)

In [ ]:
df = df_prepared

The reviews_per_month column still had some null values so I nuked these.

In [ ]:
df = df.dropna(subset=["reviews_per_month"])
df = df.reset_index(drop=True)

This code was meant to drop a couple of outliers with unrepresentative prices. I later discovered this to be doing nothing because the prices (like everything else) were already scaled. When I moved this code up and the two listings with price > 1500 were dropped, the opposite of what I expected happened. My models performed 50-100% worse. In theory, unrepresentative training data is bad because it causes bias. I was confused by this but couldn't justify making worse predictions so I left the two outliers alone.

In [ ]:
# print(df.shape[0])
# for x in range(df.shape[0]):
#     if df["price"][x] > 1500:
#         df = df.drop(x)       
# df = df.reset_index(drop=True)
# print(df.shape[0])

I thought it would be a good idea to compare feature importances. There is more than one way to do this but I used RandomForestRegressor's feature_importances_ attribute.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=42, n_estimators=100)
regr.fit(df.drop(["price"], axis=1), df["price"])
for x in range(len(list(df.drop(["price"], axis=1)))):
    print(list(df.drop(["price"], axis=1))[x] + " " + str(regr.feature_importances_[x]))

Unsurprisingly, bedrooms and beds are the most important features. Shared room is the least important feature because it only applies to less than 0.1% of the listings. I was a bit surprised that accommodates and bathrooms weren't more important. I was even more surprised that House, Apartment and Other were basically a waste of time. I could and, in hindsight, should have seen this coming by doing some visualisation to compare the prices of houses and apartments. Rather than going back and pretending I did, I'll fess up to this mistake.

## Creating Training and Test Sets

Evaluating models on training data alone leads to biased results. To avoid this trap, I split the data into training and test sets. What this means is that models can be trained on 80% of the data and tested on the other 20%. I also dropped price from both sets into test_set_labels and train_set_labels.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
test_set.head()
train_set.head()

test_set_labels = test_set["price"].copy()
test_set = test_set.drop("price", axis=1)
train_set_labels = train_set["price"].copy()
train_set = train_set.drop("price", axis=1)

## Linear Regression

First, I used was linear regression. This is, as far as I know, the most basic regression model. It tries to find the line of best fit through the data. I trained it on the training set and tried it on a few examples from the test set.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train_set, train_set_labels)

some_data = test_set.iloc[:5]
some_labels = test_set_labels.iloc[:5]

print("Predictions:", lin_reg.predict(some_data))

print("Labels:", list(some_labels))

I calculated the root mean square error.

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

predictions = lin_reg.predict(test_set)

lin_mse = mean_squared_error(test_set_labels, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

I also used cross validation.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_reg, test_set, test_set_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

The error was lower in 8 of the 10 cases, suggesting this train/test split was "unlucky".

## Decision Tree Regression

Second, I used decision tree regression, so named because it uses a decision tree to decide what it thinks a listing's price is based on its other features.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(train_set, train_set_labels)
tree_predict = tree_reg.predict(test_set)

print(np.sqrt(mean_squared_error(test_set_labels, tree_predict)))

I adjusted max_depth by trial and error and found a value of 4 to be optimal...

In [ ]:
tree_reg = DecisionTreeRegressor(max_depth=4, random_state=42)
tree_reg.fit(train_set, train_set_labels)
tree_predict = tree_reg.predict(test_set)

print(np.sqrt(mean_squared_error(test_set_labels, tree_predict)))

... and used cross validation.

In [ ]:
scores = cross_val_score(tree_reg, test_set, test_set_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)
print(np.mean(rmse_scores))

## Support Vector Regression

Third, I used support vector regression.

In [ ]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(train_set, train_set_labels)
svm_predict = svm_reg.predict(test_set)

print(np.sqrt(mean_squared_error(test_set_labels, svm_predict)))

I adjusted epsilon by trial and error and found a value of 1.5 to be optimal...

In [ ]:
svm_reg = LinearSVR(epsilon=0.4, random_state=42)
svm_reg.fit(train_set, train_set_labels)
svm_predict = svm_reg.predict(test_set)

print(np.sqrt(mean_squared_error(test_set_labels, svm_predict)))

... and used cross validation.

In [ ]:
scores = cross_val_score(svm_reg, test_set, test_set_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

## Random Forest Regression

Fourth, I used random forest regression. A random forest is an ensemble of decision trees which introduces extra randomness. 

In [ ]:
rf_reg = RandomForestRegressor(max_depth=2, random_state=42, n_estimators=100)
rf_reg.fit(train_set, train_set_labels)
predictions = rf_reg.predict(test_set)

print(np.sqrt(mean_squared_error(test_set_labels, predictions)))

I adjusted max_dep and n_estimators by trial and error and found values of 3 and 200 to be optimal...

In [ ]:
rf_reg = RandomForestRegressor(max_depth=3, random_state=42, n_estimators=200)
rf_reg.fit(train_set, train_set_labels)
predictions = rf_reg.predict(test_set)

print(np.sqrt(mean_squared_error(test_set_labels, predictions)))

... and used cross validation.

In [ ]:
scores = cross_val_score(rf_reg, test_set, test_set_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)
print(np.mean(rmse_scores))

Something that surprised me was that despite my optimising both, the random forest regressor did not beat the lone decision tree regressor. This is why cross validation matters. Comparing the cross validation scores of the decision tree and random forest regressors, I found the random forest regressor to be better in 9 of the 10 cases and have a significantly better mean (0.285 compared to 0.356).

With that qualification in mind, here are the scores.

| Regression     | RMSE  |
| -------------- | ----- |
| Decision Tree  | 0.287 |
| Random Forest  | 0.297 |
| Support Vector | 0.307 |
| Linear         | 0.332 |

In conclusion, all four worked quite well. Maybe the difference would have been bigger if the data set was bigger or more time was spent experimenting with the paramaters of the more complex models. This assignment helped me to understand not only the four different types of regression I used to solve the problem but also how to prepare data for them. There are some things I would have done differently and some things I would have done in a different order. I did like Jupyter Notebook a lot so hopefully I have cause to use it again in the future.